## Preparing the data : Monolingual data (MLM) and/or Parallel data (TLM)

In [10]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME 

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [27]:
%env PARA=True          
%env MONO=True           
                   
%env PARA_PATH=data/enfrde/para      
%env MONO_PATH=data/enfrde/mono    
%env SAME_VOCAB=True    
%env nCodes=10
%env shuf_n_samples=1
%env threads_for_tokenizer=16
%env test_size=10       
%env val_size=10        

# tools paths
%env TOKENIZE=tools/tokenizer_our.sh
%env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
%env FASTBPE=tools/fastBPE/fast


%env OUTPATH=data/enfrde/processed 
# create output path
! mkdir -p $OUTPATH

! chmod +x $FASTBPE
! chmod +x ../build_meta_data.sh
! chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

%env n_samples=-1

%env sub_task=en-fr:10,en-de:-1,de-fr:-1

env: PARA=True
env: MONO=True
env: PARA_PATH=data/enfrde/para
env: MONO_PATH=data/enfrde/mono
env: SAME_VOCAB=True
env: nCodes=10
env: shuf_n_samples=1
env: threads_for_tokenizer=16
env: test_size=10
env: val_size=10
env: TOKENIZE=tools/tokenizer_our.sh
env: LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
env: FASTBPE=tools/fastBPE/fast
env: OUTPATH=data/enfrde/processed
env: n_samples=-1
env: sub_task=en-fr:10,en-de:-1,de-fr:-1


In [33]:
! ../build_meta_data.sh $sub_task $n_samples 

dir data/enfrde/processed/fine_tune already exists
params ok !
*** Cleaning and tokenizing en-fr data ... ***
file data/enfrde/para/en-fr.en.all already exists
file data/enfrde/para/en-fr.fr.all already exists
*** Cleaning and tokenizing en-de data ... ***
file data/enfrde/para/en-de.en.all already exists
file data/enfrde/para/en-de.de.all already exists
*** Cleaning and tokenizing de-fr data ... ***
file data/enfrde/para/de-fr.de.all already exists
file data/enfrde/para/de-fr.fr.all already exists
file data/enfrde/mono/en.all already exists
file data/enfrde/mono/fr.all already exists
file data/enfrde/mono/en.all already exists
file data/enfrde/mono/de.all already exists
file data/enfrde/mono/de.all already exists
file data/enfrde/mono/fr.all already exists


*** split into train / valid / test ***
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write 

## Trainig

In [37]:
%env exp_id=maml
%env lgs=en-fr|en-de|de-fr

%env batch_size=2
%env max_epoch=1
%env epoch_size=100

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=80
%env valid_n_samples=10
%env test_n_samples=10
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: exp_id=maml
env: lgs=en-fr|en-de|de-fr
env: batch_size=2
env: max_epoch=1
env: epoch_size=100
env: train_n_samples=80
env: valid_n_samples=10
env: test_n_samples=10


### Pretrain a language model (MLM + TLM)

In [35]:
# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [39]:
! pwd

/home/jupyter/meta_XLM/XLM


In [41]:
%env mlm_steps=en,fr,en-fr|en,de|de,fr,de-fr

! python train.py --exp_name mlm_enfrde --exp_id $exp_id --dump_path ./dumped/ --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: mlm_steps=en,fr,en-fr|en,de|de,fr,de-fr
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm
INFO - 05/14/20 03:03:43 - 0:00:00 - ============ Initialized logger ============
INFO - 05/14/20 03:03:43 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: [[], [], []]
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 2
                                     beam_size: 1
                                     bptt: 256
                                     bt_src_langs: []
      